**Python with genomic data**

Codes for practice use

****

In [ ]:
#Reading a FASTA File

#let's say we have a FASTA file named 'myfile.fa'

f = open('myfile.fa', 'r')
seqs = {}
for line in f:
  line = line.strip()
  if line[0] == '>':
    name = line[1:]
    seqs[name] = ''
  else:
    seqs[name] += line

print(seqs)

In [ ]:
# Running blast

Import Bio

from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

fasta_string = open("test.fasta").read()
fasta_string

results = NCBIWWW.qblast("blastn", "nt", fasta_string)
blast_record = NCBIXML.read(results)

len(blast_record)

evalue_threshold = 0.001
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < evalue_threshold:
          print('****Alignment****')
          print('sequence:', alignment.title)
          print('length:', alignment.length)
          print('e value:', hsp.expect)
          print(hsp.query[0:75] + '...')


In [12]:
#Example

!pip install biopython
import Bio

from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# The unknown DNA sequence
sequence = """TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTAC
AATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCAC
CTACGGTAGAG"""

# Run BLAST
result_handle = NCBIWWW.qblast("blastn", "nt", sequence)

# Save the result to a file
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()

# Parse the BLAST results
result_handle = open("my_blast.xml")
blast_record = NCBIXML.read(result_handle)

# Extract and print the species with the lowest E-value
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < 0.01:  # Usually a threshold for significant hits
            print(f"****Alignment****")
            print(f"sequence: {alignment.title}")
            print(f"length: {alignment.length}")
            print(f"e value: {hsp.expect}")
            print(f"species: {alignment.hit_def.split('[')[-1].split(']')[0]}")
            break
result_handle.close()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.3 MB/s eta 0:00:00


KeyboardInterrupt: 

In [ ]:
#Translating into proteins

from Bio.Seq import Seq

# The DNA sequence
dna_sequence = """TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTAC
AATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCAC
CTACGGTAGAG"""

# Create the Seq object
dna_seq = Seq(dna_sequence.replace("\n", ""))

# Translate the DNA sequence to a protein sequence
protein_seq = dna_seq.translate()

# Print the protein sequence
print(protein_seq)


**Final project code**

Goals:

1.Find the numbers of record in FASTA file ('>")

2.Count the length of the sequences

3.There are 6 possible reading frames, open reading frame is called ORF.
 1. What is the length of the longtest ORF?
 2. What is the identifier?

4.Find the numbers of repeats in forward string, given length n


In [14]:
#Load required packages

from Bio import SeqIO
from Bio.Seq import Seq
import re
from collections import defaultdict, Counter


In [15]:
#Create a function to count # of records in FASTA file
def count_records(filename):
    return sum(1 for record in SeqIO.parse(filename, "fasta"))

In [16]:
#Create a function to count the len of the sequences

def sequence_lengths(filename):
    lengths = {}
    for record in SeqIO.parse(filename, "fasta"):
        lengths[record.id] = len(record.seq)
    max_length = max(lengths.values())
    min_length = min(lengths.values())
    longest_sequences = [id for id, length in lengths.items() if length == max_length]
    shortest_sequences = [id for id, length in lengths.items() if length == min_length]
    return lengths, max_length, min_length, longest_sequences, shortest_sequences


In [17]:
#Assess open reading frame

def find_orfs(sequence, frame):
    sequence = sequence[frame-1:]  # Adjust for reading frame
    orfs = []
    start_positions = [m.start() for m in re.finditer('ATG', str(sequence))]
    stop_codons = {'TAA', 'TAG', 'TGA'}

    for start in start_positions:
        for pos in range(start + 3, len(sequence), 3):
            codon = sequence[pos:pos+3]
            if codon in stop_codons:
                orfs.append((start + frame, sequence[start:pos+3]))  # Store 1-based position
                break

    return orfs

def longest_orf_in_file(filename, frame):
    max_orf_length = 0
    max_orf_seq_id = ""
    max_orf_seq = ""
    max_orf_start_pos = 0

    for record in SeqIO.parse(filename, "fasta"):
        orfs = find_orfs(record.seq, frame)
        for start_pos, orf in orfs:
            orf_length = len(orf)
            if orf_length > max_orf_length:
                max_orf_length = orf_length
                max_orf_seq_id = record.id
                max_orf_seq = orf
                max_orf_start_pos = start_pos

    return max_orf_length, max_orf_seq_id, max_orf_seq, max_orf_start_pos


In [18]:
#Find all repeats

def find_repeats(sequence, length):
    repeats = defaultdict(int)
    for i in range(len(sequence) - length + 1):
        repeat = sequence[i:i+length]
        repeats[repeat] += 1
    return repeats

def find_most_frequent_repeat(filename, repeat_length):
    all_repeats = Counter()
    for record in SeqIO.parse(filename, "fasta"):
        repeats = find_repeats(str(record.seq), repeat_length)
        all_repeats.update(repeats)

    most_common_repeat = all_repeats.most_common(1)[0]
    return most_common_repeat, all_repeats


In [21]:
#Results

#Read in fasta file

f = open('dna2.fasta', 'r')

!ls

dna2.fasta  sample_data


In [22]:
# Count number of records in FASTA file

print(count_records('dna2.fasta'))

#18

18


In [26]:
# Count the length of the longest sequences

print(sequence_lengths('dna2.fasta'))


({'gi|142022655|gb|EQ086233.1|91': 4635, 'gi|142022655|gb|EQ086233.1|304': 1151, 'gi|142022655|gb|EQ086233.1|255': 4894, 'gi|142022655|gb|EQ086233.1|45': 3511, 'gi|142022655|gb|EQ086233.1|396': 4076, 'gi|142022655|gb|EQ086233.1|250': 2867, 'gi|142022655|gb|EQ086233.1|322': 442, 'gi|142022655|gb|EQ086233.1|88': 890, 'gi|142022655|gb|EQ086233.1|594': 967, 'gi|142022655|gb|EQ086233.1|293': 4338, 'gi|142022655|gb|EQ086233.1|75': 1352, 'gi|142022655|gb|EQ086233.1|454': 4564, 'gi|142022655|gb|EQ086233.1|16': 4804, 'gi|142022655|gb|EQ086233.1|584': 964, 'gi|142022655|gb|EQ086233.1|4': 2095, 'gi|142022655|gb|EQ086233.1|277': 1432, 'gi|142022655|gb|EQ086233.1|346': 115, 'gi|142022655|gb|EQ086233.1|527': 2646}, 4894, 115, ['gi|142022655|gb|EQ086233.1|255'], ['gi|142022655|gb|EQ086233.1|346'])


In [28]:
#What is the length of the longest ORF appearing in reading frame 2 of any of the sequences?

print(longest_orf_in_file('dna2.fasta', 2))

def longest_orf_in_frame_2(filename):
    max_orf_length = 0
    max_orf_seq_id = ""

    for record in SeqIO.parse(filename, "fasta"):
        orfs = find_orfs(record.seq, 2)
        for start_pos, orf in orfs:
            orf_length = len(orf)
            if orf_length > max_orf_length:
                max_orf_length = orf_length
                max_orf_seq_id = record.id

    return max_orf_length, max_orf_seq_id
print(longest_orf_in_frame_2('dna2.fasta'))

(2394, 'gi|142022655|gb|EQ086233.1|45', Seq('ATGGAGAAACAGTCTCGCGTTACGCGCGACGGTCGCGGGAGAGTTCTATGCGGT...TGA'), 385)
(2394, 'gi|142022655|gb|EQ086233.1|45')


In [29]:
#What is the starting position of the longest ORF in reading frame 3 in any of the sequences?

def longest_orf_in_frame_3(filename):
    max_orf_length = 0
    max_orf_seq_id = ""
    max_orf_start_pos = 0

    for record in SeqIO.parse(filename, "fasta"):
        orfs = find_orfs(record.seq, 3)
        for start_pos, orf in orfs:
            orf_length = len(orf)
            if orf_length > max_orf_length:
                max_orf_length = orf_length
                max_orf_seq_id = record.id
                max_orf_start_pos = start_pos + 1  # Convert to 1-based index

    return max_orf_length, max_orf_seq_id, max_orf_start_pos

print(longest_orf_in_frame_3('dna2.fasta'))


(2394, 'gi|142022655|gb|EQ086233.1|45', 386)


In [30]:
# Find repeats

def most_frequent_repeat(filename, repeat_length):
    all_repeats = Counter()
    for record in SeqIO.parse(filename, "fasta"):
        repeats = find_repeats(str(record.seq), repeat_length)
        all_repeats.update(repeats)

    most_common_repeat, count = all_repeats.most_common(1)[0]
    return most_common_repeat, count

print(most_frequent_repeat('dna2.fasta', 6))

('GCGCGC', 153)


In [34]:
# find how many 12 repeats occur

def all_repeats_with_max_count(filename, repeat_length):
    all_repeats = Counter()
    for record in SeqIO.parse(filename, "fasta"):
        repeats = find_repeats(str(record.seq), repeat_length)
        all_repeats.update(repeats)

    max_count = max(all_repeats.values())
    max_repeats = [seq for seq, count in all_repeats.items() if count == max_count]
    return max_count, max_repeats

print(all_repeats_with_max_count('dna2.fasta', 12))

def main(filename):
    repeat_length = 12
    max_count, max_repeats = all_repeats_with_max_count(filename, repeat_length)
    print(f"The maximum number of copies (Max) of any 12-base repeat is: {max_count}")
    print(f"Number of different 12-base sequences that occur {max_count} times: {len(max_repeats)}")
    print(f"These sequences are: {max_repeats}")

if __name__ == "__main__":
    filename = "dna2.fasta"
    main(filename)
print(main('dna2.fasta'))

(10, ['CATTCGCCATTC', 'ATTCGCCATTCG', 'TTCGCCATTCGC', 'TCGCCATTCGCC'])
The maximum number of copies (Max) of any 12-base repeat is: 10
Number of different 12-base sequences that occur 10 times: 4
These sequences are: ['CATTCGCCATTC', 'ATTCGCCATTCG', 'TTCGCCATTCGC', 'TCGCCATTCGCC']
The maximum number of copies (Max) of any 12-base repeat is: 10
Number of different 12-base sequences that occur 10 times: 4
These sequences are: ['CATTCGCCATTC', 'ATTCGCCATTCG', 'TTCGCCATTCGC', 'TCGCCATTCGCC']
None


In [37]:
# length  = 7, which repreat occurs the most

print(most_frequent_repeat('dna2.fasta', 7))

def main(filename):
    repeat_length = 7
    max_count, max_repeats = all_repeats_with_max_count(filename, repeat_length)
    print(f"The maximum number of copies (Max) of any 12-base repeat is: {max_count}")
    print(f"Number of different 12-base sequences that occur {max_count} times: {len(max_repeats)}")
    print(f"These sequences are: {max_repeats}")

if __name__ == "__main__":
    filename = "dna2.fasta"
    main(filename)
print(main('dna2.fasta'))

('CGCGCCG', 63)
The maximum number of copies (Max) of any 12-base repeat is: 63
Number of different 12-base sequences that occur 63 times: 1
These sequences are: ['CGCGCCG']
The maximum number of copies (Max) of any 12-base repeat is: 63
Number of different 12-base sequences that occur 63 times: 1
These sequences are: ['CGCGCCG']
None
